In [8]:
#Import Libraries
from bs4 import BeautifulSoup
from html2text import html2text
import pandas as pd
import pprint as pp
import requests
import time

In [9]:
#Extract Job Title Function
def extract_job_title_from_result(soup): 
    jobs = []
    for div in soup.find_all(name = "div", attrs = {"class":"row"}):
        for a in div.find_all(name = "a", attrs = {"data-tn-element":"jobTitle"}):
            jobs.append(a["title"])
    return(jobs)

In [10]:
#Extract Job URL Function
def extract_job_url_from_result(soup): 
    urls = []
    for div in soup.find_all(name = "div", attrs = {"class":"row"}):
        for a in div.find_all(name = "a", attrs = {"data-tn-element":"jobTitle"}):
            this_url = a['href']
            to_go_url = "https://www.indeed.com/viewjob" + this_url[7:]
            urls.append(to_go_url)
    return(urls)

In [11]:
def extract_text_from_jobURL(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.text, "html.parser")
    
    table = soup.find('table', id = 'job-content')
    span = table.find('span', id = 'job_summary')
    div = span.find('div')
    
    return(html2text(str(div)))

In [13]:
#Import the Job Title Dataset
job_titles=pd.read_csv("..\Datasets\Job_Titles.csv", sep = ",")

#Create job_titles list
job_list=list(job_titles.Job_titles)

#Replace spaces with '+'
job_list2=[]
for item in job_list:
    job_list2.append(item.replace(' ', '+'))
    
job_list=job_list2

In [58]:
#Create URLs (for testing)
short_job_list=['Top+Executives', 'Chief+Executives']
short_city_list=['New+York','Chicago','San+Francisco']


url_list=[]
for city in short_city_list:
    for job in short_job_list:
        for start in range(0,20,10):
            URL="http://www.indeed.com/jobs?q="+ job + "%2420%2C000&l=" + str(city) + "&start=" + str(start)
            url_list.append(URL)

In [59]:
columns=["job_title","job_description"]
df = pd.DataFrame(columns = columns)

for i in url_list:

    #Conducting a request of the stated URL above:
    page = requests.get(i)
    
    #Specifying a desired format of “page” using the html parser
    soup = BeautifulSoup(page.text, "html.parser")

    #Go to job page
    url_list = extract_job_url_from_result(soup)
    
    #Extract job title
    job_title=extract_job_title_from_result(soup)
    
    #Extract job description
    job_description=extract_text_from_jobURL(url_list[0])
    
    #Add a new row
    df.loc[len(df)]=[job_title,job_description]

In [60]:
df.head()

,job_title,job_description
0,"[Administrative Assistant, Tower Technician / ...","We are PepsiCo Sales. We are game changers, mo..."
1,"[VP Event Presentation, Associate Occupancy Pl...",## **Overview **\n\nThe Madison Square Garden ...
2,"[Chief of Staff, Chief of Staff, Chief of Staf...",None\n\n
3,"[Chief of Staff to the Provost, Associate - Of...",The Chief of Staff to the Provost works as a c...
4,"[Receptionist, Help Desk Specialist, Mental He...","## **Overview **\n\nARCO/Murray is a premier, ..."
